In [5]:
# 必要なインポートを実施
# スクレイピング用フレームワーク
from bs4 import BeautifulSoup
# リクエスト取得ライブラリ
import requests
# お馴染みPandas
import pandas as pd
# お馴染みSeriesとDataFrame
from pandas import Series,DataFrame
from datetime import datetime as dt
# 正規表現ライブラリ
import re
# データフレームを綺麗に表示させるためのライブラリ
from IPython.display import display, HTML

In [38]:
# 競馬情報の取得をWebスクレイピングで実施
HOME_URL = 'https://www.nankankeiba.com/'
PLACE = '川崎'

# URLからコンテンツを取得する
def url_to_soup(url):
    req = requests.get(url)
    return BeautifulSoup(req.content, 'html.parser')

# 各馬の過去１０レースリンクを取得(出走表のページを設定)
def horse_page_link(url):
    soup = url_to_soup(url)
    link_list = [HOME_URL + x.get('href') for x in soup.find_all('a', class_='tx-mid tx-low') ]
    return link_list

hors_page_link_list = horse_page_link(HOME_URL + 'race_info/2018082321060403.do')
print(len(hors_page_link_list))

12


In [39]:
# HTMLのタグを排除する正規表現
p = re.compile(r"<[^>]*?>")

tag_to_text = lambda x: p.sub("", x).split('\n') 
split_tr = lambda x: str(x).split('</tr>')

# tableタグを取得し、trタグでsqlitする
def get_previous_race_row(soup):
    race_table = soup.select("table.tb01")[2]
    return [tag_to_text(x)  for x in split_tr(race_table)]

# 各馬の過去10レースを取得し、データフレームに入れ込む
def horse_data(url):
    soup = url_to_soup(url)

    # 過去のレースデータ
    pre_race_data = get_previous_race_row(soup)
    df = pd.DataFrame(pre_race_data)[1:][[2,3,10,11,13,14,15,19,23]].dropna().rename(columns={
        2:'date', 3:'place', 10:'len', 11:'wether', 13:'popularity', 14:'rank', 15:'time',19:'weight',23:'money'})
    horse_name = soup.find('h2', id='tl-prof').get_text()
    return horse_name, df


In [40]:
# 馬場状態のカラム内容を文字列によって変更する
def add_soil_columns(row):
        row['soil_heavy'] = 1 if row['wether'][-2:] =='/重'  else 0
        row['soil_s_heavy'] = 1 if row['wether'][-2:] =='稍重'  else 0
        row['soil_good'] = 1 if row['wether'][-2:] =='/良'  else 0
        row['soil_bad'] = 1 if row['wether'][-2:] =='不良'  else 0
        return row

# レースデータのカラムを加工
def add_race_data(df):
    df_ =pd.DataFrame()
    for idx, row in df.iterrows():
        if row['popularity'] == '':
            continue

        # 馬場状態
        row = add_soil_columns(row)

        row['money']=int(row['money'].replace(',','')) 
        row['horse_cnt'] = int(row['rank'].split('/')[1])
        row['result_rank'] = int(row['rank'].split('/')[0])
        row['len'] = int(row['len'][0:4])
        row['popularity'] = int(row['popularity'])
        row['weight'] = int(row['weight'])

        # 　競馬場の一致
        row['same_place'] = 1 if row['place'].startswith(PLACE)  else 0

        # タイム(秒)
        try:
            time = dt.strptime(row['time'], '%M:%S.%f')
            row['sec'] = time.minute*60 + time.second + time.microsecond/1000000 
        except ValueError:
            time = dt.strptime(row['time'], '%S.%f')
            row['sec'] = time.second + time.microsecond/1000000

        row['sec'] = int(row['sec']) 

        df_ = df_.append(row, ignore_index=True)
    return df_

df_list = []
# 取得した出走馬の過去レースをデータフレームに格納
for url_link in hors_page_link_list:
    name, df = horse_data(url_link)
    print(name)
    df = add_race_data(df)
    display(df)
    df_list.append(df)
    

コンスタンツェ


,date,horse_cnt,len,money,place,popularity,rank,result_rank,same_place,sec,soil_bad,soil_good,soil_heavy,soil_s_heavy,time,weight,wether
0,18/8/23,12.0,900.0,0.0,川崎,6.0,7/12,7.0,1.0,57.0,0.0,1.0,0.0,0.0,57.6,529.0,晴/良
1,18/7/20,12.0,900.0,135000.0,川崎,8.0,4/12,4.0,1.0,56.0,0.0,1.0,0.0,0.0,56.8,515.0,晴/良
2,18/7/2,12.0,900.0,0.0,川崎☆,11.0,8/12,8.0,1.0,57.0,0.0,1.0,0.0,0.0,57.5,503.0,晴/良
3,18/6/13,10.0,1400.0,0.0,川崎,8.0,10/10,10.0,1.0,100.0,0.0,0.0,1.0,0.0,1:40.7,501.0,晴/重
4,17/10/11,10.0,1400.0,0.0,川崎,2.0,8/10,8.0,1.0,94.0,0.0,0.0,0.0,1.0,1:34.6,520.0,曇/稍重
5,17/8/22,10.0,1500.0,160000.0,川崎,5.0,3/10,3.0,1.0,99.0,0.0,0.0,0.0,1.0,1:39.6,523.0,曇/稍重
6,17/8/1,11.0,1400.0,0.0,川崎,3.0,8/11,8.0,1.0,97.0,0.0,0.0,1.0,0.0,1:37.0,521.0,雨/重
7,17/7/24,10.0,1500.0,0.0,船橋,5.0,9/10,9.0,0.0,103.0,0.0,1.0,0.0,0.0,1:43.9,515.0,曇/良
8,17/7/4,11.0,1400.0,96000.0,川崎,4.0,5/11,5.0,1.0,94.0,0.0,1.0,0.0,0.0,1:34.2,523.0,曇/良
9,17/6/23,9.0,1500.0,280000.0,船橋,7.0,2/9,2.0,0.0,100.0,0.0,0.0,1.0,0.0,1:40.8,517.0,晴/重


コウギョウセイント


,date,horse_cnt,len,money,place,popularity,rank,result_rank,same_place,sec,soil_bad,soil_good,soil_heavy,soil_s_heavy,time,weight,wether
0,18/8/23,12.0,900.0,0.0,川崎,11.0,11/12,11.0,1.0,58.0,0.0,1.0,0.0,0.0,58.6,492.0,晴/良
1,18/7/2,12.0,900.0,0.0,川崎☆,12.0,12/12,12.0,1.0,58.0,0.0,1.0,0.0,0.0,58.0,499.0,晴/良
2,18/6/12,11.0,1400.0,0.0,川崎☆,10.0,11/11,11.0,1.0,99.0,0.0,0.0,1.0,0.0,1:39.3,502.0,晴/重
3,18/3/1,11.0,900.0,0.0,川崎,11.0,11/11,11.0,1.0,58.0,1.0,0.0,0.0,0.0,58.4,506.0,晴/不良
4,17/10/10,9.0,1400.0,0.0,川崎☆,7.0,9/9,9.0,1.0,95.0,0.0,0.0,0.0,1.0,1:35.8,496.0,晴/稍重
5,17/9/7,12.0,900.0,0.0,川崎,12.0,12/12,12.0,1.0,59.0,0.0,0.0,1.0,0.0,59.1,495.0,曇/重
6,17/4/3,10.0,900.0,0.0,川崎,9.0,10/10,10.0,1.0,59.0,0.0,0.0,0.0,1.0,59.4,473.0,晴/稍重
7,17/3/1,10.0,900.0,0.0,川崎,10.0,10/10,10.0,1.0,57.0,0.0,1.0,0.0,0.0,57.3,468.0,晴/良
8,17/1/2,12.0,900.0,0.0,川崎,9.0,10/12,10.0,1.0,57.0,0.0,1.0,0.0,0.0,57.1,475.0,晴/良
9,16/12/16,9.0,1500.0,0.0,川崎☆,9.0,9/9,9.0,1.0,104.0,0.0,0.0,1.0,0.0,1:44.5,480.0,晴/重


キョウエイスパーク


...

,date,horse_cnt,len,money,place,popularity,rank,result_rank,same_place,sec,soil_bad,soil_good,soil_heavy,soil_s_heavy,time,weight,wether
0,18/8/23,12.0,900.0,108000.0,川崎,5.0,5/12,5.0,1.0,56.0,0.0,1.0,0.0,0.0,56.7,491.0,晴/良
1,18/7/20,12.0,900.0,0.0,川崎,6.0,7/12,7.0,1.0,57.0,0.0,1.0,0.0,0.0,57.1,483.0,晴/良
2,18/7/2,12.0,900.0,0.0,川崎☆,8.0,9/12,9.0,1.0,57.0,0.0,1.0,0.0,0.0,57.5,495.0,晴/良
3,16/5/23,12.0,900.0,0.0,川崎☆,5.0,7/12,7.0,1.0,56.0,0.0,1.0,0.0,0.0,56.4,483.0,晴/良
4,16/5/9,11.0,900.0,0.0,川崎☆,1.0,7/11,7.0,1.0,56.0,0.0,0.0,0.0,1.0,56.0,487.0,雨/稍重
5,16/4/1,12.0,900.0,0.0,川崎☆,3.0,7/12,7.0,1.0,57.0,0.0,1.0,0.0,0.0,57.1,483.0,曇/良
6,16/3/3,11.0,900.0,900000.0,川崎,3.0,1/11,1.0,1.0,55.0,0.0,1.0,0.0,0.0,55.7,489.0,晴/良
7,16/1/25,12.0,900.0,0.0,川崎,2.0,7/12,7.0,1.0,57.0,0.0,1.0,0.0,0.0,57.0,494.0,晴/良
8,16/1/4,12.0,900.0,270000.0,川崎,4.0,2/12,2.0,1.0,55.0,0.0,1.0,0.0,0.0,55.6,490.0,晴/良
9,15/10/22,12.0,900.0,90000.0,川崎,4.0,5/12,5.0,1.0,57.0,0.0,1.0,0.0,0.0,57.0,480.0,晴/良


ラルグス


,date,horse_cnt,len,money,place,popularity,rank,result_rank,same_place,sec,soil_bad,soil_good,soil_heavy,soil_s_heavy,time,weight,wether
0,18/8/23,12.0,900.0,135000.0,川崎,1.0,4/12,4.0,1.0,56.0,0.0,1.0,0.0,0.0,56.4,482.0,晴/良
1,18/7/20,12.0,900.0,342000.0,川崎,2.0,2/12,2.0,1.0,56.0,0.0,1.0,0.0,0.0,56.4,473.0,晴/良
2,18/2/26,12.0,900.0,900000.0,川崎,2.0,1/12,1.0,1.0,55.0,0.0,1.0,0.0,0.0,55.9,479.0,曇/良
3,18/1/1,12.0,1400.0,112000.0,川崎,1.0,4/12,4.0,1.0,94.0,0.0,1.0,0.0,0.0,1:34.6,476.0,晴/良
4,17/12/14,11.0,1400.0,0.0,川崎,1.0,6/11,6.0,1.0,93.0,0.0,1.0,0.0,0.0,1:33.1,483.0,晴/良
5,17/2/18,16.0,1400.0,0.0,京都,6.0,9/16,9.0,0.0,86.0,0.0,0.0,0.0,1.0,1:26.8,468.0,曇/稍重
6,17/2/5,11.0,1600.0,600000.0,東京,5.0,5/11,5.0,0.0,101.0,0.0,1.0,0.0,0.0,1:41.6,476.0,曇/良


テネレッツァ


,date,horse_cnt,len,money,place,popularity,rank,result_rank,same_place,sec,soil_bad,soil_good,soil_heavy,soil_s_heavy,time,weight,wether
0,18/8/23,12.0,900.0,900000.0,川崎,2.0,1/12,1.0,1.0,55.0,0.0,1.0,0.0,0.0,55.4,436.0,晴/良
1,18/7/19,12.0,1400.0,200000.0,川崎,4.0,3/12,3.0,1.0,92.0,0.0,1.0,0.0,0.0,1:32.1,436.0,晴/良
2,18/2/1,11.0,1400.0,160000.0,川崎,3.0,3/11,3.0,1.0,92.0,0.0,0.0,0.0,1.0,1:32.3,419.0,雨/稍重
3,18/1/3,11.0,1400.0,112000.0,川崎,4.0,4/11,4.0,1.0,94.0,0.0,1.0,0.0,0.0,1:34.6,418.0,晴/良
4,17/12/11,12.0,1500.0,0.0,川崎,6.0,6/12,6.0,1.0,100.0,0.0,1.0,0.0,0.0,1:40.6,427.0,晴/良
5,17/11/8,12.0,1200.0,40000.0,門別,3.0,2/12,2.0,0.0,75.0,0.0,0.0,1.0,0.0,1:15.4,430.0,雨/重
6,17/11/1,7.0,1200.0,30000.0,門別,4.0,3/7,3.0,0.0,74.0,1.0,0.0,0.0,0.0,1:14.5,436.0,雨/不良
7,17/10/19,9.0,1200.0,30000.0,門別,1.0,3/9,3.0,0.0,76.0,0.0,0.0,0.0,1.0,1:16.5,436.0,晴/稍重
8,17/10/5,10.0,1200.0,40000.0,門別,3.0,2/10,2.0,0.0,77.0,0.0,1.0,0.0,0.0,1:17.2,430.0,晴/良
9,17/9/20,9.0,1200.0,40000.0,門別,3.0,2/9,2.0,0.0,76.0,0.0,0.0,1.0,0.0,1:16.3,434.0,曇/重


In [ ]:
# 該当のレース結果データを取得
def result_data(url):
    soup = url_to_soup(url)

    # 土の状態
    condition = soup.find(id="race-data02").get_text().replace('\n','').split(';')[1].split('　')[2][0:2]

    # レースの長さ
    race_len = int(soup.find(id="race-data01-a").get_text().replace('\n','').split('　')[3].replace(',','')[1:4])

    # 1位の馬番
    p = re.compile('<td class="al-center">')
    hukusyo_list = []
    hukusyo_list.append(int(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[2]).replace('\n','') ))

    # レース日
    race_date_str = soup.find(id="race-data01-a").get_text().replace('\n','').split(';')[0].split('日')[0]
    race_date = dt.strptime(race_date_str, '%Y年%m月%d')
    return hukusyo_list, condition, race_len, race_date

a, b, c, d = result_data('https://www.nankankeiba.com/result/2018082321060403.do')
print(d)
df = horse_data('https://www.nankankeiba.com/result/2018082321060403.do', d)
df = add_race_data(df)